In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb

### 1. Carregamento dos Dados

In [ ]:
dataset_features = pd.read_csv("/content/drive/MyDrive/dataset-processado/dataset_features.csv")
dataset_features

,Unnamed: 0,Hora Início,Dia da Semana,Total Domicílios | Rat%,AB | Rat%,C1 | Rat%,C2 | Rat%,DE | Rat%,Masculino | Rat%,Feminino | Rat%,...,Categoria_REALITY SHOW,Categoria_RELIGIOSO,Categoria_REPORTAGEM,Categoria_RURAL,Categoria_SERIES,Categoria_SHOW,Categoria_SORTEIO,Categoria_TELE VENDAS,Feriado,Ano
0,0,6.00,1,4.27,2.40,0.31,0.82,5.23,1.65,2.35,...,0,0,0,0,0,0,0,0,0,2020
1,1,6.00,1,4.89,3.02,0.51,1.08,5.23,1.89,2.75,...,0,0,0,0,0,0,0,0,0,2020
2,2,6.00,1,5.74,3.13,1.53,1.22,5.23,1.97,3.24,...,0,0,0,0,0,0,0,0,0,2020
3,3,6.25,1,6.55,3.06,1.86,1.48,6.46,1.92,3.92,...,0,0,0,0,0,0,0,0,0,2020
4,4,6.25,1,7.27,3.49,2.18,1.48,6.46,2.23,4.05,...,0,0,0,0,0,0,0,0,0,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218863,218863,29.50,4,1.67,0.30,0.41,1.03,0.53,0.17,0.92,...,0,0,0,0,0,0,0,0,0,2022
218864,218864,29.50,4,1.97,0.30,0.41,1.45,0.35,0.17,1.07,...,0,0,0,0,0,0,0,0,0,2022
218865,218865,29.75,4,2.22,0.30,0.41,1.45,0.88,0.17,1.26,...,0,0,0,0,0,0,0,0,0,2022
218866,218866,29.75,4,2.71,0.54,0.41,1.81,0.88,0.37,1.42,...,0,0,0,0,0,0,0,0,0,2022


### 2. Seleção das Features e Labels

Nessa parte definimos as varíaveis dependentes (y) e variáveis indepentes (x) que foram utilizadas nos modelos de predição. 

In [ ]:
from sklearn.model_selection import train_test_split

# features a serem utilizadas
X = dataset_features[['Hora Início','Dia da Semana','Mês','Dia', 'Feriado', 'Categoria_CARROS E MOTORES', 'Categoria_CULINARIO', 'Categoria_DEBATE',  
       'Categoria_DOCUMENTARIO', 'Categoria_EDUCATIVO', 'Categoria_ENTREVISTA',
       'Categoria_ESPORTE', 'Categoria_FEMININO', 'Categoria_FILME',
       'Categoria_FUTEBOL', 'Categoria_GAME SHOW', 'Categoria_HUMORISTICO',
       'Categoria_JORNALISMO', 'Categoria_MINISSERIE', 'Categoria_MUSICAL',
       'Categoria_NOVELA', 'Categoria_OUTROS', 'Categoria_POLITICO',
       'Categoria_PREMIACAO', 'Categoria_REALITY SHOW', 'Categoria_RELIGIOSO',
       'Categoria_REPORTAGEM', 'Categoria_RURAL', 'Categoria_SERIES',
       'Categoria_SHOW', 'Categoria_SORTEIO', 'Categoria_TELE VENDAS']]

X_sem_categoria = dataset_features[['Hora Início','Dia da Semana','Mês','Dia', 'Feriado']]

X_somente_categoria = dataset_features[['Categoria_CARROS E MOTORES', 'Categoria_CULINARIO', 'Categoria_DEBATE',  
       'Categoria_DOCUMENTARIO', 'Categoria_EDUCATIVO', 'Categoria_ENTREVISTA',
       'Categoria_ESPORTE', 'Categoria_FEMININO', 'Categoria_FILME',
       'Categoria_FUTEBOL', 'Categoria_GAME SHOW', 'Categoria_HUMORISTICO',
       'Categoria_JORNALISMO', 'Categoria_MINISSERIE', 'Categoria_MUSICAL',
       'Categoria_NOVELA', 'Categoria_OUTROS', 'Categoria_POLITICO',
       'Categoria_PREMIACAO', 'Categoria_REALITY SHOW', 'Categoria_RELIGIOSO',
       'Categoria_REPORTAGEM', 'Categoria_RURAL', 'Categoria_SERIES',
       'Categoria_SHOW', 'Categoria_SORTEIO', 'Categoria_TELE VENDAS']]


# labels a serem utilizadas
y_rat = dataset_features[['Total Domicílios | Rat%', 'AB | Rat%',
       'C1 | Rat%', 'C2 | Rat%', 'DE | Rat%', 'Masculino | Rat%',
       'Feminino | Rat%', '4-11 anos | Rat%', '12-17 anos | Rat%',
       '18-24 anos | Rat%', '25-34 anos | Rat%', '35-49 anos | Rat%',
       '50-59 anos | Rat%', '60+ anos | Rat%']]

# TODO: Change to fid and share columns
y_fid = dataset_features[[
       'Total Indivíduos | Fid%', 'AB | Fid%', 'C1 | Fid%', 'C2 | Fid%',
       'DE | Fid%', 'Masculino | Fid%', 'Feminino | Fid%', '4-11 anos | Fid%',
       '12-17 anos | Fid%', '18-24 anos | Fid%', '25-34 anos | Fid%',
       '35-49 anos | Fid%', '50-59 anos | Fid%', '60+ anos | Fid%']]

y_fid['Total Indivíduos | Fid%'] = y_fid['Total Indivíduos | Fid%'].str.replace(',','.').astype('float64')

y_share = dataset_features[['Total Domicílios | Shr%', 'AB | Shr%', 'C1 | Shr%', 'C2 | Shr%',
       'DE | Shr%', 'Masculino | Shr%', 'Feminino | Shr%', '4-11 anos | Shr%',
       '12-17 anos | Shr%', '18-24 anos | Shr%', '25-34 anos | Shr%',
       '35-49 anos | Shr%', '50-59 anos | Shr%', '60+ anos | Shr%']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### 2.1 Separação entre features/labels de teste e features/labels de validação

In [ ]:
X_train, X_test, y_train_rat, y_test_rat = train_test_split(X,y_rat, random_state=42)
_, _, y_train_share, y_test_share = train_test_split(X,y_share, random_state=42)
_, _, y_train_fid, y_test_fid = train_test_split(X,y_fid, random_state=42)

### 3. Modelos de Regressão
import das bibliotecas necessárias

In [ ]:
# TODO: Importar aqui somente os modelos e ferramentas utilizadas

# Modelos
from lightgbm import LGBMRegressor

# Métricas de Avaliação
from sklearn.metrics import r2_score, mean_squared_error

#Facilitador para múltiplos outputs
from sklearn.multioutput import MultiOutputRegressor

### 3.1 Light GBM

In [ ]:
modelo_lgbm_rat = MultiOutputRegressor(LGBMRegressor(learning_rate=0.04, num_leaves=247, random_state=42, n_estimators=10000, colsample_bytree=0.9982550687656891, subsample=0.5512789317708288, verbosity=-1, num_threads=0, min_sum_hessian_in_leaf=5.0760508277840675, reg_alpha=0.0000017987930473761658
, reg_lambda=0.05702044679692809))
modelo_lgbm_share = MultiOutputRegressor(LGBMRegressor(learning_rate=0.04, num_leaves=248, random_state=42, n_estimators=10000, colsample_bytree=0.9493232832624843, subsample=0.8454217478794726, verbosity=-1, num_threads=0, min_sum_hessian_in_leaf=0.08096469608219674, reg_alpha=0.0000017287289950264267
, reg_lambda=0.002173930392411803))
modelo_lgbm_fid = MultiOutputRegressor(LGBMRegressor(learning_rate=0.04, num_leaves=208, random_state=42, n_estimators=585, colsample_bytree=0.8747240161725821, subsample=0.765298451176346, verbosity=-1, num_threads=0, min_sum_hessian_in_leaf=0.004211470867172107, reg_alpha=0.014838817621637291, reg_lambda=0.0000022946640156968945))

#Rat
modelo_lgbm_rat.fit(X_train,y_train_rat,verbose=2,eval_metric='logloss')

#Share
modelo_lgbm_share.fit(X_train,y_train_share,verbose=2,eval_metric='logloss')

#Fid
modelo_lgbm_fid.fit(X_train,y_train_fid,verbose=2,eval_metric='logloss')


MultiOutputRegressor(estimator=LGBMRegressor(colsample_bytree=0.8747240161725821,
                                             learning_rate=0.04,
                                             min_sum_hessian_in_leaf=0.004211470867172107,
                                             n_estimators=585, num_leaves=208,
                                             num_threads=0, random_state=42,
                                             reg_alpha=0.014838817621637291,
                                             reg_lambda=2.2946640156968945e-06,
                                             subsample=0.765298451176346,
                                             verbosity=-1))

In [ ]:
# Rat Scores
test_pred_lgbm_rat = modelo_lgbm_rat.predict(X_test)
train_pred_lgbm_rat = modelo_lgbm_rat.predict(X_train)
print("\n\n## Rat %")
print('Precisão (treino): ',r2_score(y_train_rat, train_pred_lgbm_rat))
print('Mean Squared Error (Treino): ', mean_squared_error(y_train_rat, train_pred_lgbm_rat))
print('Precisão (teste): ', r2_score(y_test_rat, test_pred_lgbm_rat))
print('Mean Squared Error (teste): ', mean_squared_error(y_test_rat, test_pred_lgbm_rat))

# Share Scores
test_pred_lgbm_share = modelo_lgbm_share.predict(X_test)
train_pred_lgbm_share = modelo_lgbm_share.predict(X_train)
print("\n\n## Share %")
print('Precisão (treino): ',r2_score(y_train_share, train_pred_lgbm_share))
print('Mean Squared Error (Treino): ', mean_squared_error(y_train_share, train_pred_lgbm_share))
print('Precisão (teste): ', r2_score(y_test_share, test_pred_lgbm_share))
print('Mean Squared Error (teste): ', mean_squared_error(y_test_share, test_pred_lgbm_share))

# Fid Scores
test_pred_lgbm_fid = modelo_lgbm_fid.predict(X_test)
train_pred_lgbm_fid = modelo_lgbm_fid.predict(X_train)
print("\n\n## Fid %")
print('Precisão (treino): ',r2_score(y_train_fid, train_pred_lgbm_fid))
print('Mean Squared Error (Treino): ', mean_squared_error(y_train_fid, train_pred_lgbm_fid))
print('Precisão (teste): ', r2_score(y_test_fid, test_pred_lgbm_fid))
print('Mean Squared Error (teste): ', mean_squared_error(y_test_fid, test_pred_lgbm_fid))



### 3.1.1 Testes de Hiperparâmetros

###### 3.1.1.1 Importação de Bibliotecas

In [ ]:
!pip install --quiet optuna
!pip install verstack
import optuna
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

In [ ]:
from verstack import LGBMTuner

###### 3.1.1.2 Teste de Hiperparâmetro para Rat

In [ ]:
tuner_rat = LGBMTuner(metric = 'mse')
tuner_rat.fit(X_train, y_train_rat['Total Domicílios | Rat%'])
pred_rat = tuner_rat.predict(X_test)

###### 3.1.1.3 Teste de Hiperparâmetro para Share

In [ ]:
tuner_share = LGBMTuner(metric = 'mse')
tuner_share.fit(X_train, y_train_share['Total Domicílios | Shr%'])
pred_share = tuner_share.predict(X_test)

###### 3.1.1.4 Teste de Hiperpatrâmetro para Fid

In [ ]:
tuner_fid = LGBMTuner(metric = 'mse')
tuner_fid.fit(X_train, y_train_fid['Total Indivíduos | Fid%'])
pred_fid = tuner_fid.predict(X_test)

In [ ]:
tuner.plot_importances(interactive = True)